In [1]:
import datetime
import re

from bs4 import BeautifulSoup
import requests

In [2]:
def get_category_urls(category_url,start_page='',limit=None):
    result = []; count = 0
    page_url = category_url+'&p='+str(start_page)
    while True:
        print(page_url)
        soup = BeautifulSoup(requests.get(page_url).text,'html.parser')
        urls = ['http://www.setn.com'+soup.select("ol a")[i].attrs['href'] for i in range(len(soup.select("ol a")))]
        result.extend(urls)
        next_link = soup.find_all("div", attrs={'class' : 'pager'})[0].find_all('a')[-1]['href']
        if page_url=='http://www.setn.com'+next_link or len(urls)==0:
            break
            
        count += 1
        if limit!=None and count >= limit:
            break
            
        page_url = 'http://www.setn.com'+next_link
    return result

In [3]:
def parser_page(url):
    """
    parser page method 
    主要用於抓取新聞detail頁面資訊

    page_data = parser_page(news detail url)
    """
    soup = BeautifulSoup(requests.get(url).text,'html.parser')
    search = lambda typ,seed: soup.find_all("meta", attrs={typ : seed})[0].attrs['content']
    inputs = [("property","og:url"),("property","og:title"),("name","pubdate"),("name","Keywords"),("name","section")]
    
    parse = [search(typ,seed) for typ,seed in inputs]
    post_time = datetime.datetime.strptime(parse[2], '%Y-%m-%dT%H:%M:%S')
    
    article = soup.select("article div > p")    
    content = []; journalist = None
    for i in range(len(article)):
        if len(article[i].attrs) == 0 and not journalist:
            journalist = article[i].text
        else:
            content.append(article[i].text)   
    content = "".join(content)
    
    fb_url = 'https://graph.facebook.com/fql?q=SELECT%20like_count,%20total_count,%20share_count,'+ \
              '%20click_count,%20comment_count%20FROM%20link_stat%20WHERE%20url%20=%20%22'+parse[0]+'%22'

    res_data = requests.get(fb_url).json()['data'][0]
    fb_like = res_data['like_count']
    fb_share = res_data['share_count']
    fb_com = res_data['comment_count']
    
    ######facebook comments#######   
    tree = {}
    fb_com_url = 'http://graph.facebook.com/comments?id=' + parse[0]+ \
                 '&limit=100&filter=stream&fields=parent.fields%28id%29,message,from,like_count,created_time,parent'
    fb_data = requests.get(fb_com_url).json()
    while True:
        if len(fb_data["data"])==0:
            break
        for comment in fb_data['data']:
            one_com = {}
            one_com["post_time"] = datetime.datetime.strptime(comment['created_time'][:-5], '%Y-%m-%dT%H:%M:%S')
            one_com["actor"] = comment['from']['name']
            one_com["like"] = comment['like_count']
            one_com["content"] = comment['message']
            one_com["source_type"] = "facebook"
            one_com["dislike"] = None
            if 'parent' in comment:
                tree[comment['parent']['id']]["sub_comments"].append(one_com)
            else:
                one_com["sub_comments"] = []
                tree[comment['id']] = one_com

        fb_next_url = fb_com_url+'&after='+fb_data['paging']['cursors']['after']
        fb_data = requests.get(fb_next_url).json()
    ######facebook comments#######        
    
    return {
        "url": parse[0],
        "source_press": None,
        "title": parse[1],
        "post_time": post_time,
        "journalist": journalist,
        "content": content,
        "compare": None,
        "keyword": parse[3].split(','),
        "fb_like": fb_like,
        "fb_share": fb_share,
        "fb_comment_count": fb_com,
        "category": parse[4].split(','),
        "comment": [tree[key] for key in sorted(tree.keys())]
    }

In [51]:
collect = {}
def collect_resp(url):
    collect[url] = old_req(url)
    return collect[url]

old_req = requests.get
requests.get = collect_resp
#requests.get = old_req

In [54]:
def find_start_url(category_url, min_step = 3, init = 32):
    step = init; now = 1+int(init/2)
    page_url = category_url+'&p='+str(now)
    while True:
        soup = BeautifulSoup(requests.get(page_url).text,'html.parser')
        num_urls = len(soup.select("ol a"))
        if step/2 <= min_step:
            break
        if num_urls==0:
            step = int(step/2)
            now -= step           
        else:
            step = int(step/2) if step != init else init
            now += step 
        page_url = category_url+'&p='+str(now)
    
    ans = now-step-2 if num_urls==0 else now-2
    return 1 if ans < 1 else ans

N = 40
cate_urls = {}
group_id = [2,4,5,6,7,8,9,15,17,18,19,31,34,41,42]
for c in group_id:
    url = 'http://www.setn.com/ViewAll.aspx?PageGroupID='+str(c)
    cate_urls[c] = get_category_urls(url,find_start_url(url))[-N:]

http://www.setn.com/ViewAll.aspx?PageGroupID=2&p=23
http://www.setn.com/ViewAll.aspx?pagegroupid=2&p=24
http://www.setn.com/ViewAll.aspx?pagegroupid=2&p=25
http://www.setn.com/ViewAll.aspx?pagegroupid=2&p=26
http://www.setn.com/ViewAll.aspx?pagegroupid=2&p=27
http://www.setn.com/ViewAll.aspx?PageGroupID=4&p=111
http://www.setn.com/ViewAll.aspx?pagegroupid=4&p=112
http://www.setn.com/ViewAll.aspx?pagegroupid=4&p=113
http://www.setn.com/ViewAll.aspx?pagegroupid=4&p=114
http://www.setn.com/ViewAll.aspx?PageGroupID=5&p=71
http://www.setn.com/ViewAll.aspx?pagegroupid=5&p=72
http://www.setn.com/ViewAll.aspx?pagegroupid=5&p=73
http://www.setn.com/ViewAll.aspx?pagegroupid=5&p=74
http://www.setn.com/ViewAll.aspx?PageGroupID=6&p=163
http://www.setn.com/ViewAll.aspx?pagegroupid=6&p=164
http://www.setn.com/ViewAll.aspx?pagegroupid=6&p=165
http://www.setn.com/ViewAll.aspx?pagegroupid=6&p=166
http://www.setn.com/ViewAll.aspx?PageGroupID=7&p=7
http://www.setn.com/ViewAll.aspx?pagegroupid=7&p=8
http:/

In [66]:
result = {}
for g in group_id:
    result[g] = [parser_page(page_url) for page_url in cate_urls[g]]
       

In [81]:
import pickle

with open('target.pickle', 'wb') as f:
    pickle.dump((cate_urls,result), f, pickle.HIGHEST_PROTOCOL)

In [82]:
with open('target.pickle', 'rb') as f:
    data = pickle.load(f)

In [112]:
with open('resps.pickle', 'wb') as f:
    pickle.dump(collect, f, pickle.HIGHEST_PROTOCOL)

In [7]:
#Facebook comment example
example = 'http://www.setn.com/News.aspx?NewsID=117306&PageGroupID=6&ProjectID=987'
tree = {}
fb_com_url = 'http://graph.facebook.com/comments?id=http://www.setn.com/News.aspx?NewsID='+ \
              re.findall('NewsID=([0-9]+)',example)[0]+ \
             '&limit=100&filter=stream&fields=parent.fields%28id%29,message,from,like_count,created_time,parent'


fb_data = requests.get(fb_com_url).json()
while True:
    if len(fb_data["data"])==0:
        break
    for comment in fb_data['data']:
        one_com = {}
        one_com["post_time"] = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',comment['created_time'][:-5])))
        one_com["actor"] = comment['from']['name']
        one_com["like"] = comment['like_count']
        one_com["content"] = comment['message']
        one_com["source_type"] = "facebook"
        one_com["dislike"] = None
        if 'parent' in comment:
            tree[comment['parent']['id']]["sub_comments"].append(one_com)
        else:
            one_com["sub_comments"] = []
            tree[comment['id']] = one_com
    
    fb_next_url = fb_com_url+'&after='+fb_data['paging']['cursors']['after']
    fb_data = requests.get(fb_next_url).json()


In [ ]:
import pprint
pprint.pprint(tree)

In [1]:
#Test
import requests
import pickle
import datetime

from bs4 import BeautifulSoup
from setn_parser import get_category_urls, parser_page

with open('target.pickle', 'rb') as f:
    data = pickle.load(f)
    
with open('resps.pickle', 'rb') as f:
    resps = pickle.load(f)
    

def test_requests(url):
    return resps[url]

def find_start_url(category_url, min_step = 3, init = 32):
    step = init; now = 1+int(init/2)
    page_url = category_url+'&p='+str(now)
    while True:
        soup = BeautifulSoup(requests.get(page_url).text,'html.parser')
        num_urls = len(soup.select("ol a"))
        if step/2 <= min_step:
            break
        if num_urls==0:
            step = int(step/2)
            now -= step           
        else:
            step = int(step/2) if step != init else init
            now += step 
        page_url = category_url+'&p='+str(now)
    
    ans = now-step-2 if num_urls==0 else now-2
    return 1 if ans < 1 else ans


def test_parser_page(group_id, N=40):
    old_req = requests.get
    requests.get = test_requests
    
    result = {}; target = {}
    for gid in group_id:
        result[gid] = [parser_page(page_url) for page_url in data[0][gid][-N:]]
        target[gid] = data[1][gid][-N:]
    
    requests.get = old_req
    assert result == target, "..."
    

def test_get_category_urls(group_id, N=40):
    old_req = requests.get
    requests.get = test_requests
    
    result = {}; target = {}
    for gid in group_id:
        url = 'http://www.setn.com/ViewAll.aspx?PageGroupID='+str(gid)
        result[gid] = get_category_urls(url,find_start_url(url))[-N:]
        target[gid] = data[0][gid][-N:]
    requests.get = old_req

    assert result == target, "..."

test_parser_page([4,5,6,7,8],100)
test_get_category_urls([4,5,6,7,8],10)